# Exercise: Implementing Motion Detection Using Background Subtraction

### Objective

In this exercise, you will implement motion detection in a video using background subtraction. Follow the steps outlined below and write the corresponding code for each step.

### Step 1: Import Required Libraries

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
import numpy as np
import cv2


### Step 2: Set Up Video Capture

Initialize the video capture object by loading the video file (e.g., `video.mp4`). This object will allow you to read frames from the video.

In [17]:
cap = cv2.VideoCapture('/content/drive/MyDrive/Datasets/Computer Vision/Copy of Motion_Detection_Test.mp4')

### Step 3: Define the Video Writer

Set up a `VideoWriter` object to save the processed video with motion detection. Choose the appropriate codec (e.g., `'mp4v'`) and specify the frame rate and resolution for the output video.

In [18]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('motion_output.mp4', fourcc, 20.0, (600, 500))

### Step 4: Create the Background Subtractor

Use OpenCV's MOG2 background subtractor to create a background model that will help detect moving objects. Set `detectShadows=True` to improve detection accuracy by accounting for shadows.

In [19]:
fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows=True)

### Step 5: Process the Video Frame by Frame

1. In a loop, capture each frame from the video.
2. Resize the frame for consistency.
3. Apply the background subtractor to detect moving objects in the frame.
4. Create a binary thresholded image to isolate the moving objects.
5. Apply morphological operations like erosion and dilation to reduce noise and strengthen the detected areas.
6. Detect contours (boundaries) of the moving objects.
7. For each contour, calculate its area and filter out small movements by considering only contours with areas greater than a certain threshold (e.g., 1200).
8. Draw rectangles around the detected moving objects and annotate the motion.

In [22]:
while True:
    success, img = cap.read()

    if success:
      img = cv2.resize(img,(600, 500))
      fgmask = fgbg.apply(img)
      _, thresh = cv2.threshold(fgmask.copy(), 180, 255, cv2.THRESH_BINARY)

      kernel = np.ones((5,5), np.uint8)
      thresh = cv2.erode(thresh, kernel) #takes image and kernel
      thresh = cv2.dilate(thresh, None, iterations=6)

      contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

      for contour in contours:
            area = cv2.contourArea(contour)

            if area > 1200:
                x, y, w, h = cv2.boundingRect(contour)

                cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 255), 3)
                cv2.putText(img, 'MOTION DETECTED', (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

                out.write(img)

    else:
        # Breakif no more frames are available
        break




### Step 6: Release Resources

After processing all the frames, release the video capture and writer objects to free up system resources. Close any OpenCV windows that were opened during the process.

In [23]:
cap.release()
out.release()

cv2.destroyAllWindows()


### Bonus Challenge

Try experimenting with different threshold values, kernel sizes, or codecs to optimize the motion detection and improve the output video quality.